In [1]:
# -*- coding: utf-8 -*-
import sys
import codecs
import json

In [2]:
import requests
from bs4 import BeautifulSoup
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse as urlparse

In [3]:
link = "https://batdongsan.vn/"
t = "/ban-nha/p"
r = requests.get(link+t[0])
soup = BeautifulSoup(r.text, 'html.parser')
# print(r)

In [4]:
def find_link(soup):
    l = []
    div_items = soup.find_all("div", attrs={"class" : "item"})
    for div_item in div_items:
        a_links = div_item.find_all('a',href=True)
        for a_link in a_links:
            l.append(a_link['href'])
    return l

# for value in find_link(soup):
#     print(value)

In [5]:
def find_data(soup):
    data = {}

    # Tiêu đề
    titles = soup.find_all('div', {"class": "name"})
    data['title'] = titles[0].find('a').text.strip() if titles else None

    # Giá
    prices = soup.find_all('span', {"class": "price"})
    data['price'] = prices[0].text.strip() if prices else None

    # Diện tích
    acreages = soup.find_all('span', {"class": "acreage"})
    data['acreage'] = acreages[0].text.strip() if acreages else None

    # Mô tả
    descriptions = soup.find_all('div', {"class": "sapo"})
    data['description'] = descriptions[0].find('p').text.strip() if descriptions else None

    # Ảnh
    images = soup.find_all('div', {"class": "image cover"})
    image_tag = images[0].find('img') if images else None
    data['image_url'] = image_tag['src'] if image_tag and 'src' in image_tag.attrs else None

    # Link sản phẩm
    link_tags = images[0].find_all('a') if images else None
    data['product_link'] = link_tags[0]['href'] if link_tags else None

    return data


def process_property_elements(soup):
    data_list = []

    # Tìm tất cả các phần tử HTML chứa thông tin
    property_elements = soup.find_all('div', {"class": "item"})

    for property_element in property_elements:
        # Gọi hàm find_data cho mỗi phần tử và thêm kết quả vào danh sách
        data_list.append(find_data(property_element))

    return data_list

def scrape_and_save(link, base_link, t, max_pages):
    all_data = []

    for i in range(1, max_pages + 1):
        try:
            r = requests.get(link + t + str(i))
            r.raise_for_status()  # Raise HTTPError for bad requests
            soup = BeautifulSoup(r.text, 'html.parser')
            
            property_data = process_property_elements(soup)
            all_data.extend(property_data)

        except requests.RequestException as e:
            print(f"Error fetching page {i}: {e}")

    with open('bat_dong_san_vn.json', 'w') as file:
        json.dump(all_data, file)

    print("Done")

In [6]:
max_pages = 50

scrape_and_save(link, link, t, max_pages)

Done
